[<img src="../assets/images/Logo.png" width="64" align="center">](https://github.com/a-fuchs/jupyter-micropython-extended-kernel) &emsp;  [README](../README.md) / [NoteBooks](./Index.ipynb) | **Uart REPL**

---

# Uart REPL

In this sample a HC-05 classic BlueTooth modul is conneced to a ESp32 on pin 16 (TX) and 17 (RX).  
**Attention!** Connect *tx* off controller to *rx* of HC-05 and *rx* of controller to *tx* of HC-5.

If all is set up, a connection to the controller could be established with the apropriate port, for example:

```
%serialconnect --port=/dev/rfcomm0 --baud=9600
```
or for example:
```
%serialconnect --port=COM4 --baud=9600
```


---

## HC-0X BlueTooth UART
---

    Speed:     9600 bps
    Data bits: 8 bits
    Parity:    None
    Stop bits: 1 bit

    Pincode 1234


* [HC-05 Datasheet](https://solderingmind.com/hc-05-datasheet/)


**Windows:**

Pair the device and then use the corresponding COM-Port.

**Linux:**

As root: bind HC-05/HC-06 to `/dev/rfcomm0`

```bash
rfcomm bind rfcomm0 00:23:11:01:33:D2
```

Test on Terminal as user:
```bash
picocom --b 9600 /dev/rfcomm0
```

Look, what happens on the port:<br>
#interceptty -s 'ispeed 115200 ospeed 115200' -l /dev/ttyUSB0 /dev/ttyDUMMY | interceptty-nicedump  
interceptty -s 'ispeed 115200 ospeed 115200' -l /dev/ttyUSB0 /dev/ttyDUMMY

---

## Copy moduls to the controller

In [ ]:
%serialconnect

---
### class REPLManager

This class is copied to the controller so we can use it with `import replmanager`.

In [ ]:
%sendtofile "/replmanager.py"
#%%writefile replmanager.py
from machine import UART, Pin, Timer
import os
from time import sleep

class REPLManager:
    timer       = None
    mapUart     = {}
    
    # Attention! Connect *tx* off controller to *rx* of HC-05 and *rx* of controller to *tx* of HC-5.
    # BAUD_RATE = 9600  2400  4800  9600  19200  38400  57600  115200
    @staticmethod
    def setUartREPL( id = 1, baudrate = 9600, tx = 16, rx = 17 ):
        REPLManager.stopTimer()

        uart = REPLManager.mapUart.get( id )
        
        if uart is None:
            uart = UART( id )
            REPLManager.mapUart[ id ] = uart
            
        uart.init( baudrate = baudrate, tx = tx, rx = rx )
        
        # Found at: https://github.com/micropython/micropython/issues/6023            
        # https://github.com/micropython/micropython/issues/6023#issuecomment-2466170834
        # <a href="https://github.com/robert-hh">Robert Hammelrath</a>
        # "It seems that dupterm is not told that data is incoming.
        # You can do so using [...] after enabling dupterm."
        os.dupterm( uart )
        try:
            REPLManager.timer = Timer( -1, mode=Timer.PERIODIC, period=20, callback=os.dupterm_notify )
        except:
            REPLManager.timer = Timer( 0, mode=Timer.PERIODIC, period=20, callback=os.dupterm_notify )
  
        # 'uart.irq' does not exist 2025-02-26:
        # h05BlueToothREPL.uart.irq(trigger=UART.IRQ_RX, handler=os.dupterm_notify)

    @staticmethod
    def setDefaultREPL():
        REPLManager.stopTimer()
        os.dupterm( None )
        
    @staticmethod
    def stopTimer():
        if REPLManager.timer is not None:
            try:
                REPLManager.timer.deinit()
            except:
                pass
                
        REPLManager.timer = None

<hr style="color:lightblue"/>

Start Uart-REPL:

In [ ]:
# Testing:
import replmanager
replmanager.REPLManager.setUartREPL()

Reset to USB-REPL:

In [ ]:
import replmanager
replmanager.REPLManager.setDefaultREPL()

---
### main.py

The `main.py` is called every time the controller starts.  

It starts the `Uart-REPL` if the RST-Button (more precise the button defined with `_BTN_STOP`) is not pressed within 2 seconds (`_WAIT_TIME_MS=2000`), otherwise it does nothing, so you can connect over USB.

In [13]:
#%sendtofile "/main.py"
_BTN_STOP     = 0
_WAIT_TIME_MS = 2000

import time
import machine
import replmanager

btnStop = machine.Pin( _BTN_STOP, machine.Pin.IN, machine.Pin.PULL_UP )

bStarUartREPL = True
lastTimeMs    = time.ticks_ms()

while time.ticks_diff( time.ticks_ms(), lastTimeMs ) < _WAIT_TIME_MS:
    if btnStop.value() == 0:
        bStarUartREPL = False
        break
    time.sleep_ms(50)
    
if bStarUartREPL:
    try:
        replmanager.REPLManager.setUartREPL()
    except:
        pass

import gc
gc.collect()

---
## Connect to controller and test
If all is set up, a connection to the controller could be established with the apropriate port, for example:

```
%serialconnect --port=/dev/rfcomm0 --baud=9600
```
or for example:
```
%serialconnect --port=COM4 --baud=9600
```

In [ ]:
%serialconnect --port=COM4 --baud=9600

In [ ]:
print( "Hello world over classic BlueTooth!" )